# Taille des matrices


## Liste de listes

In [ ]:
l = [[1, 2, 3],
     [4, 5, 6]]

## Format dense (NumPy)

In [ ]:
import numpy as np

n = 1_000_000_000
a = np.random.randint(low=n, high=2*n, size=(30, 50))

print(a.nbytes)

12000


In [ ]:
a.dtype

dtype('int64')

In [ ]:
# calcul :
64 * 30 * 50 # bits
64 * 30 * 50 / 8 # octets

# 8 bits = 1 octet = 1 bytes

12000.0

On peut réduire la taille en changeant le type de chaque élément :

In [ ]:
a.astype('int32').nbytes

6000

On peut stocker les nombres jusqu'à :

In [ ]:
for dtype in ('int64', 'int32', 'int16'):
  print(f"{dtype} : {np.iinfo(a.astype(dtype).dtype).max}")

int64 : 9223372036854775807
int32 : 2147483647
int16 : 32767


In [ ]:
for dtype in ('int64', 'int32', 'int16'):
  print(f"{dtype} : {np.iinfo(a.astype(dtype).dtype).min}")

int64 : -9223372036854775808
int32 : -2147483648
int16 : -32768


## Format textuel

In [ ]:
np.savetxt("a.txt", a, fmt='%i')
! ls -lh a.txt

-rw-r--r-- 1 root root 17K Oct 27 10:10 a.txt


In [ ]:
! head a.txt

1759854683 1263449334 1671138775 1220098476 1085688407 1078777319 1357962593 1356228460 1931718796 1915864518 1342966990 1808003726 1144929608 1767815431 1469106986 1963102704 1721673078 1402693020 1871543596 1699144038 1475799997 1150801102 1045436975 1947320098 1863369590 1296472546 1748891290 1959861161 1756380772 1358303842 1067225329 1305745141 1234643268 1820434784 1389320056 1503340681 1841472281 1087528389 1321092885 1348738329 1734406021 1128878114 1376528439 1256142908 1291212735 1529913118 1843692210 1526635582 1916877390 1412037994
1847138785 1775123017 1506779005 1811298405 1187386942 1859074534 1871382828 1043459160 1628778047 1785507575 1440076520 1711627416 1146967913 1209264265 1800754927 1627469135 1225063513 1654242268 1061128892 1271429382 1243745794 1746774953 1475794891 1465077609 1163045872 1638126830 1910953383 1584509615 1087425882 1604968103 1349739575 1614297629 1467490630 1855751516 1045360304 1475698237 1989857303 1721555997 1094403637 1176528371 1288365452

## Format sparse (SciPy)

In [ ]:
from scipy.sparse import csr_matrix


data = np.array([5, 6, 8])
row = np.array([63, 67, 70])
col = np.array([27, 29, 35])

m = csr_matrix((data, (row, col)), shape=(3000, 4000))
m

<3000x4000 sparse matrix of type '<class 'numpy.longlong'>'
	with 3 stored elements in Compressed Sparse Row format>

À comparer avec une matrice dense :

In [ ]:
a = m.toarray()

In [ ]:
a.nbytes

96000000

# Temps de calcul

In [ ]:
x = np.zeros(shape=(3000, 4000))

## Boucle

In [ ]:
%%time

for i in range(0, 3000):
  for j in range(0, 4000):
    x[i, j] = 3 * a[i, j]

CPU times: user 7.77 s, sys: 0 ns, total: 7.77 s
Wall time: 7.78 s


## Multiplication matrice dense

In [ ]:
%%time

x = 3 * a

CPU times: user 17 ms, sys: 49 ms, total: 66 ms
Wall time: 71.6 ms


## Multiplication matrice sparse

### Cas d'une matrice pleine

In [ ]:
from scipy.sparse import random

m = random(3000, 4000, density=1)

In [ ]:
%%time

x = 3 * m

CPU times: user 83.7 ms, sys: 43.9 ms, total: 128 ms
Wall time: 132 ms


### Cas d'une matrice creuse

In [ ]:
m = random(3000, 4000, density=0.01)

In [ ]:
%%time

x = 3 * m

CPU times: user 1.16 ms, sys: 965 µs, total: 2.13 ms
Wall time: 2.16 ms


# Résumé

Il y a 3 façons principales de stocker des tableaux :
- (a) Python sous forme de liste de listes
- (b) Sous forme de matrice dense : numpy.array
- (c) Sous forme de matrice sparse (creuse) : scipy.sparse


Deux éléments importants :
- en termes de mémoire
- en terme de temps de calcul


En mémoire :
- (a) -> prend beaucoup de place parce qu'il faut stocker les headers (type d'objet, par exemple int) de chaque élément
- (b) -> prend nombre d'éléments * taille d'un élément
- (c) -> prend le nombre d'éléments non nul * taille d'un élément + nombre d'indices lignes * taille d'un indice + nombre d'indices colonnes * taille d'un indice -> environ 3 * taille d'un élément * nombre d'éléments stockés


En calcul :
- En Python : lent parce qu'une boucle Python est lente (il faut à chaque fois que Python regarde quel est le type d'objet, et se déplace dans la mémoire pour obtenir la valeur numérique correspondant à l'objet Python)
- (b) : très rapide parce que instructions en C et Fortran et utilisation optimale des caches CPU et des instruction SIMD (plusieurs calculs en une seule instruction Assembleur)
- (c) : très rapide, et surtout si peu d'éléments
